In [11]:
# ベクトル作成関数
function x_vector(xₙ, M)
    xₙₘ = zeros(length(xₙ), M)
    for m in 1:M
        xₙₘ[:,m] = xₙ.^(m-1)
    end
    xₙₘ
end

# 確認
#x_vector(1:5, 4)

# 真の次元数
M_truth = 4
W_truth = rand(-1:0.1:1,4)

# 真の入力値
X_line = range(-3, 3, length=200)
X_truth = x_vector(X_line, M_truth)

# 真の出力値
Y_line = X_truth*W_truth

# 観測データ数
N = 50
xₙ = rand(minimum(X_line):0.01:maximum(X_line),N)
xₙₘ_truth = x_vector(xₙ, M_truth)

# ノイズ成分のパラメータを設定
using Distributions
λ = 0.4
ϵ = rand(Normal(0,√inv(λ)),　N)　# 平均0ガウス分布

# 観測データの出力値
yₙ = xₙₘ_truth*W_truth + ϵ

# 作図
#using Plots
#plot(X_line, Y_line, label="Y_truth")
#scatter!(xₙ, yₙ, label="Y_obs")

# 超パラメータ
using LinearAlgebra
M = 5
mₘ = zeros(M)
Σₘₘ = Matrix{Float64}(I, M, M)*10
Λₘₘ = inv(Σₘₘ^2)

5×5 Array{Float64,2}:
 0.01  0.0   0.0   0.0   0.0
 0.0   0.01  0.0   0.0   0.0
 0.0   0.0   0.01  0.0   0.0
 0.0   0.0   0.0   0.01  0.0
 0.0   0.0   0.0   0.0   0.01

In [127]:
N = 5
X = [
    1 1 1 1
    1 2 4 8
    1 3 9 27
    1 4 16 64
]

W = rand(-1:0.1:1,4)

Yₜ = X*W

#using Plots
#plot(X[:,2], Y)

using Distributions
λ = 0.4
ϵ = rand(Normal(0,√inv(λ)),　N)


5-element Array{Float64,1}:
 -0.7160972417715565
 -2.1708205629880046
  3.404138051296579
 -1.4512053085875245
 -0.17239618398595777

In [123]:
ϵ

5-element Array{Float64,1}:
  0.20007028236073432
 -1.2279520800188344
  2.356180572436881
  4.0999661042650954
  0.17132949529221345

In [115]:
W

4-element Array{Float64,1}:
 -1.0
 -0.7
  0.1
  0.8